In [174]:
import datetime as dt
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import requests
import pandas as pd
from urllib.request import urlopen
import xml.etree.ElementTree as ET

def date_format(d):
    d = str(d).replace('-', '.')
    yyyy = int(d.split('.')[0])
    mm = int(d.split('.')[1])
    dd = int(d.split('.')[2])
    
    this_date = dt.date(yyyy, mm, dd)
    return this_date
               
            
def historical_stock_naver(stock_cd, start_date='', end_date= '', page_n = 1, last_page =0):
    
    if start_date:
        start_date = date_format(start_date)
    else:
        start_Date = dt.date.today()
    if end_date:
        end_date = date_format(end_date)
    else:
        end_date = dt.date.today()
        
    naver_stock = 'http://finance.naver.com/item/sise_day.nhn?code='+  stock_cd + '&page=' + str(page_n)
    
    source = urlopen(naver_stock).read()
    source = BeautifulSoup(source, 'lxml')
    
    dates = source.find_all('span', class_ = 'tah p10 gray03')
    prices = source.find_all('td', class_ = 'num')
    
    for n in range(len(dates)):
        
        if len(dates) >0:
            this_date = dates[n].text
            this_date = date_format(this_date)
            
            if this_date <= end_date and this_date >= start_date:
                this_close = prices[n*6].text
                this_close = this_close.replace(',', '')
                this_close = float(this_close)

                historical_prices[this_date] = this_close
            elif this_date < start_date:
                return historical_prices
            
    if last_page == 0:
        last_page = source.find_all('table')[1].find('td', class_='pgRR').find('a')['href']
        last_page = last_page.split('&')[1]
        last_page = last_page.split('=')[1]
        last_page = float(last_page)
        #last_page = 580
    
    if page_n < last_page:
        page_n = page_n + 1
        historical_stock_naver(stock_cd, start_date, end_date, page_n, last_page)
    
    return historical_prices

In [175]:
k10_historical_prices = dict()

for stock_cd in K10:
    historical_prices = dict()
    start_date = '2018-7-4'
    end_date = '2019-7-5'
    historical_prices = historical_stock_naver(stock_cd, start_date, end_date)
    
    k10_historical_prices[stock_cd] = historical_prices

In [177]:
k10_historical_price = pd.DataFrame(k10_historical_prices)
k10_historical_price.sort_index(axis=1, inplace = True)

k10_historical_price = k10_historical_price.fillna(method='ffill')
if k10_historical_price.isnull().values.any():
    k10_historical_price = k10_historical_price.fillna(method='bfill')
k10_historical_price.head()

,000660,005380,005490,005930,012330,028260,035420,051910,068270,207940
2018-07-04,85000.0,119500.0,305000.0,46250.0,198500.0,113500.0,768000.0,317500.0,291500.0,421500.0
2018-07-05,86200.0,121500.0,309000.0,45950.0,200000.0,113500.0,765000.0,320500.0,282000.0,413000.0
2018-07-06,85300.0,123500.0,312500.0,44900.0,205500.0,116000.0,749000.0,322000.0,289500.0,410000.0
2018-07-09,87600.0,124500.0,305000.0,45600.0,206000.0,119000.0,755000.0,322500.0,284000.0,427000.0
2018-07-10,87300.0,123500.0,309000.0,46300.0,205000.0,118000.0,760000.0,325500.0,281000.0,417000.0


In [187]:
historical_prices = dict()
df = pd.DataFrame()
df = k10_historical_price['005930']
df.iloc[1]

45950.0

In [162]:

samsung = pd.DataFrame.from_dict(historical_prices, orient = 'index', columns =['stock_price'])
samsung.name = 'samsung'
samsung.head()

,stock_price
2019-07-05,45450.0
2019-07-04,46000.0
2019-07-03,45400.0
2019-07-02,46250.0
2019-07-01,46600.0


In [128]:
samsung.head()

,stock_price
2019-07-05,45450.0
2019-07-04,46000.0
2019-07-03,45400.0
2019-07-02,46250.0
2019-07-01,46600.0


# 로그 함수

In [167]:
def getLogReturns(df_price):
    logReturn = np.log(df_price / df_price.shift(1))
    logReturn = logReturn.fillna(0)
    return logReturn

log_price = samsung['stock_price']
samsung['log_calc'] = getLogReturns(log_price)

samsung.head()


,stock_price,log_calc
2019-07-05,45450.0,0.000000
2019-07-04,46000.0,0.012029
2019-07-03,45400.0,-0.013129
2019-07-02,46250.0,0.018549
2019-07-01,46600.0,0.007539


# 로그 수익률 평균

In [168]:
def log_avg(dataframe):
    sum = 0
    for i in range(len(dataframe)):
        sum += dataframe.iloc[i]
    return sum / len(dataframe)

log_avg(samsung['log_calc'])
    

7.064228070828483e-05

# 표준편차 함수

In [171]:
def standard_deviation(dataframe):
    sum = 0.0
    sqrsum = 0.0
    for i in range(len(dataframe)):
        sum += (float)(dataframe['log_calc'].iloc[i])
        sqrsum += (float)(dataframe['log_calc'].iloc[i] ** 2)
        
                          
    mean = sum / len(dataframe)
    sqrmean = sqrsum / len(dataframe)
    
    stdev = sqrmean - mean * mean
    stdev = np.power(stdev, 1/2)
    
    return stdev
       
    
#standard_deviation(samsung['log_calc'])
standard_deviation(samsung)

0.015379976127506884

# 변동성함수

In [173]:
def volatility(dataframe):
    return standard_deviation(dataframe) * np.sqrt(252)

volatility(samsung)

0.24414955202096067

# 종목별 변동성